In [ ]:
import numpy as np

from ipywidgets import interact, FloatSlider, IntSlider

from bokeh.io import  output_notebook, push_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column, row, gridplot, layout
from bokeh.models import PrintfTickFormatter
from bokeh.palettes import Blues8 as palette
output_notebook(hide_banner=True)

# Course4 : Stiffness and explicit Runge Kutta methods

## Curtiss and Hirschfelder

We consider the following problem :

$$
\left\{ 
\begin{aligned} 
{\mathrm d}_t u(t) & = k \, \big(\cos(t) - u(t)) \big) \quad \text{avec } k > 1\\ 
u(t_0)  & = u_0 
\end{aligned} 
\right. 
$$

and in the following, we will assume $t_0=0$.

## Stiffness

The exact solution is given by :

$$
u(t) = \frac{k}{k^2+1} \bigg( k \cos(t) + \sin(t) \bigg) + c_0 \, e^{-k t} \quad
\text{avec} \quad c_0 = \bigg( u_0 -\frac{k}{k^2 + 1} \Big( k \cos(t_0) + \sin(t_0) \Big) \bigg)  e^{-k t_0}   
$$

In [ ]:
class curtiss_model:

    def __init__(self, k):
        self.k = k

    def fcn(self, t, u) :
        k = self.k
        u_dot = k * (np.cos(t) - u)
        return u_dot

    def sol(self, uini, t0, t):
        k = self.k

        c0 = (uini - (k/(k*k + 1)) * (k*np.cos(t0) + np.sin(t0))) * np.exp(k*t0)
        u = (k/(k*k + 1)) * (k*np.cos(t) + np.sin(t)) +  c0 * np.exp(-k*t)
        return u

In [ ]:
def show_exact_sol():

    uini = 2.
    tini = 0.
    tend = 1.5
    k = 50.
    
    cm = curtiss_model(k)
    fcn = cm.fcn
    
    texa = np.linspace(tini, tend, 500)
    uexa = cm.sol(uini, tini, texa)
 
    fig_sol = figure(x_range=(tini, tend), y_range=(0.0, 2.), plot_height=400, plot_width=800,
                     title="Exact solution of Curtiss and Hirschfelder equation")
    plt_sol = fig_sol.line(texa, uexa, color="crimson", line_width=2)
    
    show(fig_sol, notebook_handle=True)
    
    def update(k):
        cm = curtiss_model(k)
        fcn = cm.fcn
        uexa = cm.sol(uini, tini, texa)
        plt_sol.data_source.data = dict(x=texa, y=uexa)
        push_notebook()

    interact(update, k=FloatSlider(min=5, max=200, value=k, step=5))
    
show_exact_sol() 

## Runge-Kutta methods

In [ ]:
def stabiliy_disk(order):
    x = np.linspace(-4, 2, 900)
    y = np.linspace(-3, 3, 900)

    z = x + 1j*y[:, np.newaxis]
    disk = np.zeros_like(z, dtype = np.double)

     # rk1
    if order == 1:
        rk1 = z + 1
        mask = np.abs(rk1)<=1
        disk[mask] = np.abs(rk1[mask])
    # rk2
    elif order == 2:
        rk2 = z + z**2/2 + 1
        mask = np.abs(rk2)<=1
        disk[mask] = np.abs(rk2[mask])
    # rk3
    elif order == 3:
        rk3 = z + z**2/2 + z**3/6 + 1
        mask = np.abs(rk3)<=1
        disk[mask] = np.abs(rk3[mask])
    # rk4
    elif order == 4:
        rk4 = z + z**2/2 + z**3/6 + z**4/24 + 1
        mask = np.abs(rk4)<=1
        disk[mask] = np.abs(rk4[mask])
    else:
        print("order not implemented")
        exit()

    return disk 

In [ ]:
class ode_result:
    def __init__(self, y, t, nfev):
        self.y = y
        self.t = t
        self.nfev = nfev 

def forward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y[:,it+1] = yn + dt*fcn(tn, yn)

    nfev = nt-1

    return ode_result(y, t, nfev)

def rk2(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        k1 = fcn(tn, yn)
        k2 = fcn(tn + 0.5*dt, yn + dt*(0.5*k1))
        y[:,it+1] = yn + dt*k2

    nfev = 2*(nt-1)

    return ode_result(y, t, nfev)

def rk3(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        k1 = fcn(tn, yn)
        k2 = fcn(tn + 0.5*dt, yn + dt*(0.5*k1))
        k3 = fcn(tn + dt, yn + dt*(-k1 + 2*k2))
        y[:,it+1] = yn + (dt/6)*(k1+4*k2+k3)

    nfev = 3*(nt-1)

    return ode_result(y, t, nfev)

def rk4(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        k1 = fcn(tn, yn)
        k2 = fcn(tn + 0.5*dt, yn + dt*(0.5*k1))
        k3 = fcn(tn + 0.5*dt, yn + dt*(0.5*k2))
        k4 = fcn(tn + dt, yn + dt*k3)
        y[:,it+1] = yn + (dt/6)*(k1+2*k2+2*k3+k4)

    nfev = 4*(nt-1)

    return ode_result(y, t, nfev)

In [ ]:
def show_rk_sol(order):
    
    if order == 1: method = forward_euler
    elif order == 2: method = rk2    
    elif order == 3: method = rk3    
    elif order == 4: method = rk4
    else : 
        print("order not implemented")
        exit()

    uini = 2.
    tini = 0.
    tend = 1.5
    k = 50.
    
    cm = curtiss_model(k)
    fcn = cm.fcn
    
    texa = np.linspace(tini, tend, 500)
    uexa = cm.sol(uini, tini, texa)
 
    nt = 50
    dt = (tend-tini)/nt
    sol = method(tini, tend, nt, uini, fcn)
    
    uerr = np.absolute(cm.sol(uini, tini, sol.t) - sol.y[0])
    
    fig_sol = figure(x_range=(tini, tend), y_range=(-0.25, 2.25), plot_height=300, plot_width=800, title=f"Solution RK{order}")
    plt_sol = fig_sol.line(texa, uexa, color="crimson", line_width=2)
    plt_num = fig_sol.x(sol.t, sol.y[0], line_width=2, size=8)
    plt_line_num = fig_sol.line(sol.t, sol.y[0], line_width=1, line_dash="dotted")
    fig_err = figure(x_range=(tini, tend), plot_height=300, plot_width=500, title=f"Global error RK{order}")
    plt_err = fig_err.x(sol.t, uerr, line_width=2, size=8)
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%6.2e")
    
    disk = stabiliy_disk(order)
    fig_disk = figure(x_range=[-4, 2], y_range=[-3, 3], plot_width=300, plot_height=300, title=f"Stability domain RK{order}")
    fig_disk.image(image=[disk], x=-4, y=-3, dw=6, dh=6, palette=palette[::-1])
    fig_disk.segment(x0=[-4, 0], y0=[0, -4], x1=[2, 0], y1=[0, 4], color="black")
    plt_disk = fig_disk.x((-k*dt,), (0.,), color="crimson", line_width=2, size=8)
    
    show(column(fig_sol,row(fig_err, fig_disk)), notebook_handle=True)
    
    def update(nt, k):
        dt = (tend-tini)/nt
        cm = curtiss_model(k)
        fcn = cm.fcn
        uexa = cm.sol(uini, tini, texa)
        sol = method(tini, tend, nt, uini, fcn)
        uerr = np.absolute(cm.sol(uini, tini, sol.t) - sol.y[0])
        plt_sol.data_source.data = dict(x=texa, y=uexa)
        plt_num.data_source.data = dict(x=sol.t, y=sol.y[0])
        plt_line_num.data_source.data = dict(x=sol.t, y=sol.y[0])
        plt_err.data_source.data = dict(x=sol.t, y=uerr)
        plt_disk.data_source.data = dict(x=(-k*dt,), y=(0.,))
        push_notebook()

    interact(update, nt=IntSlider(min=10, max=200, value=nt, step=1, continuous_update=False),
             k=FloatSlider(min=5, max=200, value=k, step=5))
    
# Changer l'argument de la fonction suivante en 1, 2, 3 ou 4
show_rk_sol(4)